In [1]:
import sys
sys.path.insert(0, "..")
import pickle
from gpbp_osm.layers import AdmArea
from gpbp_osm import visualisation
from gpbp_osm.distance import calculate_isopolygons_graph, calculate_isopolygons_Mapbox, population_served

import osmnx as ox
import warnings
warnings.filterwarnings(action='ignore')

c:\Users\EiriniK\Documents\repos\Public-Infrastructure-Location-Optimiser\.venv\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


### Administrative Area

In [8]:
adm_area = AdmArea(country="Greece", level=2)

Retrieving data for Greece of granularity level 2
Administrative areas for level 2:
['North Aegean' 'South Aegean' 'Athos' 'Attica' 'Crete' 'Epirus'
 'West Macedonia' 'Central Macedonia' 'East Macedonia and Thrace'
 'Ionian Islands' 'Peloponnese' 'West Greece' 'Central Greece' 'Thessaly']


In [9]:
adm_area.get_adm_area(adm_name="Crete")

In [10]:
adm_area.get_facilities(method="osm", tags={"building":"hospital"})
visualisation.plot_facilities(adm_area.fac_gdf)

Retrieving {'building': 'hospital'} for Crete area


In [11]:
adm_area.get_population(method="world_pop")
#visualisation.plot_population_heatmap(adm_area.pop_df)

Data downloaded
Converting raster file to dataframe


In [5]:
adm_area.pop_df

,longitude,latitude,population
0,23.736667,35.695833,0.067376
1,23.737500,35.695833,0.070566
2,23.738333,35.695833,0.072878
3,23.735833,35.695000,0.068367
4,23.736667,35.695000,0.075395
...,...,...,...
1203050,24.121667,34.801667,0.016128
1203051,24.122500,34.801667,0.015183
1203052,24.120833,34.800833,0.016218
1203053,24.121667,34.800833,0.015266


In [5]:
adm_area.compute_potential_fac(spacing=0.05)
visualisation.plot_facilities(adm_area.grid_gdf)

Calculating isodistances for a single coordinate

In [5]:
road_network = pickle.load(open('crete_network.pickle', 'rb'))
location_coords = (24.1246768, 35.5174839)

C:\Users\EiriniK\AppData\Local\Temp\ipykernel_13576\3979363694.py:2: ResourceWarning: unclosed file <_io.BufferedReader name='crete_network.pickle'>
  road_network = pickle.load(open('crete_network.pickle', 'rb'))


In [6]:
osm_poly = calculate_isopolygons_graph(location_coords[0], location_coords[1], "length", [2000, 5000, 10000], road_network=road_network, infill=True)

In [7]:
visualisation.plot_isochrones(list(osm_poly.values()))

In [5]:
bounding_polys = calculate_isopolygons_Mapbox(
    location_coords[0], location_coords[1], "driving", "length", [2000, 5000, 10000])

In [6]:
visualisation.plot_isochrones(list(bounding_polys.values()))

Facilities - Households dictionary of population with access to hospital within 2km

In [16]:
serve_dict_mapbox = population_served(adm_area.pop_df,
    adm_area.fac_gdf,
    "facilities",
    "length",
    2000,
    "driving",
    "mapbox",
    pop_resolution=4)

In [17]:
print("Number of households (1km resolution) served per facility")
list(map(len,list(serve_dict_mapbox.values())))

Number of households (1km resolution) served per facility


[406, 525, 406, 439, 713, 453, 432, 431, 514, 260, 466]